# RunnablePassthrough

`RunnablePassthrough` serves the purpose of passing data through. This class returns **the input data as is** via the `invoke()` method.

It can be used to pass data to the next stage of the pipeline without any modification.

`RunnablePassthrough` can be useful in the following scenarios:

- When there is no need to transform or modify the data
- When a specific stage of the pipeline needs to be skipped
- When monitoring the data flow for debugging or testing purposes

This class implements the `Runnable` interface, so it can be used in the pipeline along with other `Runnable` objects.


In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    # Set a Runnable that returns the passed input as is.
    passed=RunnablePassthrough(),
    # Set a Runnable that returns the result of multiplying the "num" value of the input by 3.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # Set a Runnable that returns the result of adding 1 to the "num" value of the input.
    modified=lambda x: x["num"] + 1,
)

# Execute the Runnable with {"num": 1} as input.
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [3]:
r = RunnablePassthrough.assign(mult=lambda x: x["num"] * 3)
r.invoke({"num": 1})

{'num': 1, 'mult': 3}

*In the example above, the `passed` key is invoked with `RunnablePassthrough()`, which simply passes `{'num': 1}`.

In the second line, `RunnablePassthrough.assign` is used with a lambda function that multiplies the numeric value by 3. In this case, `extra` is set to `{'num': 1, 'mult': 3}` with the original value and an additional `mult` key.

Finally, the `modified` key is set using a lambda function that adds 1 to the `num` value, resulting in the value of the `modified` key being `2`.*

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Create a FAISS vector store from texts.
vectorstore = FAISS.from_texts(
    [
        "Teddy worked at LangChain Inc.",
        "Shirley worked at the same company as Teddy.",
        "Teddy's job is a developer.",
        "Shirley's job is a designer.",
    ],
    embedding=OpenAIEmbeddings(),
)
# Use the vector store as a retriever.
retriever = vectorstore.as_retriever()
# Define the template.
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
# Create a chat prompt from the template.
prompt = ChatPromptTemplate.from_template(template)

# Initialize the ChatOpenAI model.
model = ChatOpenAI(model_name="gpt-4o-mini")


# Function to format documents
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])


# Configure the retrieval chain.
retrieval_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [6]:
# 검색 체인을 실행하여 질문에 대한 답변을 얻습니다.
# Execute the retrieval chain to get the answer to the question.
retrieval_chain.invoke("What is Teddy's job?")

"Teddy's job is a developer."

In [7]:
# 검색 체인을 실행하여 질문에 대한 답변을 얻습니다.
retrieval_chain.invoke("What is Shirley's job?")

"Shirley's job is a designer."